In [1096]:
# Import the necessary libraries

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
#import keras

# Helper libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from os import listdir
from os.path import join
import cv2
import pandas
import os
import random
import datetime

In [1097]:
#Besoin de mettre le path absolu vers le dataset (normalement présent dans le projet)
data = "C:\\Users\\alexa\\Desktop\\M1\\Python\\Perso\\GitHub\\Projet_S1_M1\\dataset\\flower_photos"

#METTRE UNE SEULE PHOTO DANS LE DOSSIER TEST
#SINON LE MODELE FAIT DE LA MERDE
data_test = "C:\\Users\\alexa\\Desktop\\M1\\Python\\Perso\\GitHub\\Projet_S1_M1\\test"

# List out the directories inside the main input folder

folders = os.listdir(data)
folders_test = os.listdir(data_test)
print(folders)
print(folders_test)

['daisy', 'roses', 'sunflowers', 'tulips']
['random']


In [952]:
# Import the images and resize them to a 128*128 size
# Also generate the corresponding labels

image_names = []
train_labels = []
train_images = []

size = 64,64

for folder in folders:
    for file in os.listdir(os.path.join(data,folder)):
        if file.endswith("jpg"):
            image_names.append(os.path.join(data,folder,file))
            train_labels.append(folder)
            img = cv2.imread(os.path.join(data,folder,file))
            im = cv2.resize(img,size)
            train_images.append(im)
        else:
            continue
            


In [1021]:
image_names_test = []
train_labels_test = []
train_images_test = []

size = 64,64

for folder in folders_test:
    for file in os.listdir(os.path.join(data_test,folder)):
        if file.endswith("jpg"):
            image_names_test.append(os.path.join(data_test,folder,file))
            img_test = cv2.imread(os.path.join(data_test,folder,file))
            im_test = cv2.resize(img_test,size)
            train_images_test.append(im_test)
        else:
            continue


In [1022]:
cv2.imshow('image', im_test)
cv2.waitKey(0)

-1

In [1023]:
#Check if we have the correct number of photos and if the resize is correct
print(len(train_images))
print(len(train_images_test))
# Transform the image array to a numpy type and check if we have to correct data

train = np.array(train_images)
train_test = np.array(train_images_test)

print(train.shape)
print(train_test.shape)

2643
1
(2643, 64, 64, 3)
(1, 64, 64, 3)


In [852]:
# Reduce the RGB values between 0 and 1
train = train.astype('float32') / 255.0

In [853]:
# Extract the labels
label_dummies = pandas.get_dummies(train_labels)

labels =  label_dummies.values.argmax(1)

In [854]:
print(labels)

[0 0 0 ... 4 4 4]


In [855]:
print(pandas.unique(train_labels), "\n", pandas.unique(labels))

['daisy' 'dandelion' 'roses' 'sunflowers' 'tulips'] 
 [0 1 2 3 4]


In [856]:
# Shuffle the labels and images randomly for better results
union_list = list(zip(train, labels))
random.shuffle(union_list)
train,labels = zip(*union_list)

# Convert the shuffled list to numpy array type
train = np.array(train)
labels = np.array(labels)

In [1005]:
from sklearn.model_selection import train_test_split

# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(train, labels, test_size=0.25, shuffle= True)

In [1006]:
# Develop a sequential model using tensorflow keras

model = keras.Sequential([
    keras.layers.Conv2D(8,kernel_size=3,strides=2,activation='relu',padding='valid',input_shape=(64,64,3) ),
    keras.layers.Conv2D(16,kernel_size=3,strides=2,activation='relu',padding='valid' ),          
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='tanh'),
    keras.layers.Dense(5, activation='softmax')
])

In [860]:
# Compute the model parameters
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.001), #tf.train.AdamOptimizer()', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_52 (Conv2D)           (None, 31, 31, 8)         224       
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 15, 15, 16)        1168      
_________________________________________________________________
flatten_26 (Flatten)         (None, 3600)              0         
_________________________________________________________________
dense_52 (Dense)             (None, 128)               460928    
_________________________________________________________________
dense_53 (Dense)             (None, 5)                 645       
Total params: 462,965
Trainable params: 462,965
Non-trainable params: 0
_________________________________________________________________


In [933]:
# Train the model  with 5 epochs 
model.fit(x_train, y_train, epochs=5, validation_data=(x_valid, y_valid))

Train on 1982 samples, validate on 661 samples
Epoch 1/5
1982/1982 [==============================] - 2s 1ms/sample - loss: 0.5581 - accuracy: 0.8047 - val_loss: 0.5618 - val_accuracy: 0.8094
Epoch 2/5
1982/1982 [==============================] - 1s 703us/sample - loss: 0.4298 - accuracy: 0.8547 - val_loss: 0.5298 - val_accuracy: 0.8139
Epoch 3/5
1982/1982 [==============================] - 1s 701us/sample - loss: 0.3073 - accuracy: 0.9026 - val_loss: 0.5426 - val_accuracy: 0.8154
Epoch 4/5
1982/1982 [==============================] - 1s 699us/sample - loss: 0.2167 - accuracy: 0.9374 - val_loss: 0.5543 - val_accuracy: 0.7912
Epoch 5/5
1982/1982 [==============================] - 1s 710us/sample - loss: 0.1597 - accuracy: 0.9637 - val_loss: 0.5595 - val_accuracy: 0.7837


In [1082]:
model.save("model.h5")

In [1083]:
model = keras.models.load_model("model.h5")

In [1101]:
#Get the data again
img_gen_aug=keras.preprocessing.image.ImageDataGenerator(zoom_range=0.1,
                                                     rotation_range=5,
                                                     rescale=1/255,
                                                     shear_range=5,
                                                     width_shift_range=8,
                                                     height_shift_range=8,                                    
                                                     validation_split=0.1)

img_dirit=img_gen_aug.flow_from_directory(data_test,target_size=size,
                                      class_mode="sparse",
                                      batch_size=256,
                                      subset="training",
                                      interpolation="bicubic")

Found 1 images belonging to 1 classes.


In [1102]:
# Load the prediction from the model
vals=[next(img_dirit) for i in range(8)]
tx=np.concatenate([v[0] for v in vals])
ty=np.concatenate([v[1] for v in vals])
preds=model.predict(tx)

In [1103]:
i = 0
a = np.argmax(preds[i])
print("Predicted flower : ")
if a == 0:
    print("Daisy")
elif a == -1:
    print("Dandelion")
elif a == 1:
    print("Rose")
elif a == 2:
    print("Sunflower")
elif a == 3:
    print("Tulip")
    
print(preds[0])

Predicted flower : 
Tulip
[2.2819488e-01 7.5128027e-03 2.5551676e-06 7.6428974e-01]


In [1104]:
with open("model.json", "w") as file:
    file.write(model.to_json())
model.save_weights("weights.h5")

In [1105]:
# Reload the model from the 2 files we saved
with open('model.json') as json_file:
    json_config = json_file.read()
model = keras.models.model_from_json(json_config)
model.load_weights('weights.h5')

# Note that the optimizer was not preserved so we can't train the model anymore (or we have to do model.compile again).